In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from skg import nsphere_fit
import matplotlib.collections as mcoll
import matplotlib.path as mpath
import scipy.io as sio
import json
from tqdm import tqdm
import os


def colorline(
    x, y, z=None, cmap=plt.get_cmap('copper'), norm=plt.Normalize(0.0, 1.0),
        linewidth=3, alpha=1.0):
    """
    http://nbviewer.ipython.org/github/dpsanders/matplotlib-examples/blob/master/colorline.ipynb
    http://matplotlib.org/examples/pylab_examples/multicolored_line.html
    Plot a colored line with coordinates x and y
    Optionally specify colors in the array z
    Optionally specify a colormap, a norm function and a line width
    """

    # Default colors equally spaced on [0,1]:
    if z is None:
        z = np.linspace(0.0, 1.0, len(x))

    # Special case if a single number:
    if not hasattr(z, "__iter__"):  # to check for numerical input -- this is a hack
        z = np.array([z])

    z = np.asarray(z)

    segments = make_segments(x, y)
    lc = mcoll.LineCollection(segments, array=z, cmap=cmap, norm=norm,
                              linewidth=linewidth, alpha=alpha)

    ax = plt.gca()
    ax.add_collection(lc)

    return lc


def make_segments(x, y):
    """
    Create list of line segments from x and y coordinates, in the correct format
    for LineCollection: an array of the form numlines x (points per line) x 2 (x
    and y) array
    """

    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    return segments

def generate_arc(center, radius, start_angle, end_angle, num_points):
    angles = np.linspace(start_angle, end_angle, num_points)
    x = center[0] + radius * np.cos(angles)
    y = center[1] + radius * np.sin(angles)
    return x, y

# create figure and video directory
if not os.path.exists('figures'):
    os.makedirs('figures')
if not os.path.exists('videos'):
    os.makedirs('videos')

%matplotlib qt

In [3]:
prefix = '20hr-wingless-orco-tt'
FPS = 10
N_FLIES = 4

In [4]:
# PHASE 1
# load background image from mat file
p1_bg = sio.loadmat(f'{prefix}_phase_1/{prefix}_phase_1-bg.mat')['bg'][0][0].item()[0]
# PHASE 2
# load background image from mat file
p2_bg = sio.loadmat(f'{prefix}_phase_2/{prefix}_phase_2-bg.mat')['bg'][0][0].item()[0]
# PHASE 3
# load background image from mat file
p3_bg = sio.loadmat(f'{prefix}_phase_3/{prefix}_phase_3-bg.mat')['bg'][0][0].item()[0]

In [5]:
recalculate = False
if os.path.exists('arena.json') and not recalculate:
    
    with open('arena.json', 'r') as f:
        arena = json.load(f)

    center = np.array(arena['center'])
    radius = arena['radius']
    sf = arena['sf']

else:
    # get arena margins
    plt.figure()
    plt.imshow(p1_bg, cmap='gray')
    plt.title('Select 5 points on the margin of the arena')
    pts = plt.ginput(5)
    pts = np.array(pts)
    plt.close()

    # TRUE RADIUS
    true_radius = 75 # mm

    radius, center = nsphere_fit(pts)
    print('Center:', center)
    print('Radius:', radius)
    sf = radius / true_radius

    # save the properties of the arena
    arena = {
        'center': center.tolist(),
        'radius': radius,
        'sf': sf
    }

    with open('arena.json', 'w') as f:
        json.dump(arena, f)



In [6]:
def get_ring(ring_pts):
    outer_ring = ring_pts[:5]
    inner_ring = ring_pts[5:]

    # fit a circle to the points
    outer_radius, outer_center = nsphere_fit(outer_ring)
    inner_radius, inner_center = nsphere_fit(inner_ring)

    # get the average center
    center = (outer_center + inner_center) / 2

    # get the average radius
    radius = (outer_radius + inner_radius) / 2

    # get the average trail width
    trail_width = outer_radius - inner_radius

    # ring properties
    ring_props = {
        'center': center,
        'radius': radius,
        'trail_width': trail_width,
    }
    return ring_props

def draw_ring(ring_props, bg=p2_bg, axis=None, color='r'):
    # draw the estimated circles
    if axis is None:
        fig, ax = plt.subplots()
    else:
        ax = axis

    ax.imshow(bg, cmap='gray')

    circ_inner = plt.Circle(ring_props['center'], ring_props['radius']-ring_props['trail_width']/2, color=color, fill=False)
    circ_outer = plt.Circle(ring_props['center'], ring_props['radius']+ring_props['trail_width']/2, color=color, fill=False)
    ax.add_artist(circ_inner)
    ax.add_artist(circ_outer)

    if axis is None:
        ax.set_aspect('equal')
        plt.title('Estimated ring')
        plt.show()

def ring_coordinates(pos,y_props):
    # get the angle of the point
    position = np.arctan2(pos[1]-y_props['center'][1], pos[0]-y_props['center'][0])
    position = position % (2*np.pi)
    # get the distance from the center
    dist = np.sqrt((pos[0]-y_props['center'][0])**2 + (pos[1]-y_props['center'][1])**2) - y_props['radius']
    return position, dist


In [7]:
recalculate = False

# check if previous ring props are available
try:
    if recalculate:
        raise FileNotFoundError
    
    with open('big_ring_props.json', 'r') as f:
        big_ring_props = json.load(f)
        big_ring_props = {k: np.array(v) if isinstance(v, list) else v for k, v in big_ring_props.items()}

except FileNotFoundError:
    # get ring points
    plt.figure()
    plt.imshow(p1_bg, cmap='gray')
    plt.title('Pick 5 points on the outer edge of the big circle and 5 points on the inner edge of the big circle')
    big_ring_pts = plt.ginput(10)
    plt.close()

    big_ring_props = get_ring(big_ring_pts)

    # save ring props as json
    with open('big_ring_props.json', 'w') as f:
        # convert numpy arrays to lists
        big_ring_props = {k: v.tolist() if isinstance(v, np.ndarray) else v for k, v in big_ring_props.items()}
        json.dump(big_ring_props, f)

    draw_ring(big_ring_props)

# check if previous ring props are available
try:
    if recalculate:
        raise FileNotFoundError
    
    with open('small_ring_props.json', 'r') as f:
        small_ring_props = json.load(f)
        small_ring_props = {k: np.array(v) if isinstance(v, list) else v for k, v in small_ring_props.items()}
except FileNotFoundError:
    # get ring points
    plt.figure()
    plt.imshow(p3_bg, cmap='gray')
    plt.title('Pick 5 points on the outer edge of the small circle and 5 points on the inner edge of the small circle')
    small_ring_pts = plt.ginput(10)
    plt.close()

    small_ring_props = get_ring(small_ring_pts)

    # save ring props as json
    with open('small_ring_props.json', 'w') as f:
        # convert numpy arrays to lists
        small_ring_props = {k: v.tolist() if isinstance(v, np.ndarray) else v for k, v in small_ring_props.items()}
        json.dump(small_ring_props, f)

    draw_ring(small_ring_props)

In [8]:
max_displacement = 50 # mm/s 
max_displacement = max_displacement * sf / FPS

#### JUMP TEST CODE

In [ ]:
# get x and y 
for i in range(N_FLIES):
    df = pd.read_csv(f'{prefix}_phase_1/{prefix}_phase_1-trackfeat.csv/fly{i+1}.csv')
    
    x = df['pos x'].values
    y = df['pos y'].values
    # interpolate missing values
    x = np.interp(np.arange(len(x)), np.arange(len(x))[~np.isnan(x)], x[~np.isnan(x)])
    y = np.interp(np.arange(len(y)), np.arange(len(y))[~np.isnan(y)], y[~np.isnan(y)])

    # get difference between consecutive frames
    dx = np.diff(x)
    dy = np.diff(y)
    d = np.sqrt(dx**2 + dy**2)

    plt.plot(x, y, 'k-', lw=0.2)
    # mark every point where d > max_displacement
    points = np.where(d > max_displacement)[0]
    # draw a line between consecutive points
    for i in points:
        plt.plot(x[i:i+2], y[i:i+2], 'r-', lw=1)
plt.gca().set_aspect('equal')
plt.show()

### TRACKLET CONVERSION

In [9]:
# convert to tracklets
for phase in [1,2,3]:
    tracklets = []
    for i in range(N_FLIES):
        df = pd.read_csv(f'{prefix}_phase_{phase}/{prefix}_phase_{phase}-trackfeat.csv/fly{i+1}.csv')
        x = df['pos x'].values
        y = df['pos y'].values
        ori = df['ori'].values
        frame = df.index.values

        # interpolate missing values
        x = np.interp(np.arange(len(x)), np.arange(len(x))[~np.isnan(x)], x[~np.isnan(x)])
        y = np.interp(np.arange(len(y)), np.arange(len(y))[~np.isnan(y)], y[~np.isnan(y)])
        ori = np.interp(np.arange(len(ori)), np.arange(len(ori))[~np.isnan(ori)], ori[~np.isnan(ori)])
        
        # get difference between consecutive frames
        d = np.sqrt(np.diff(x)**2 + np.diff(y)**2)
        # get points where d > max_displacement
        points = np.where(d > max_displacement)[0]
        start = 0
        for i in points:
            t = pd.DataFrame({'pos x': x[start:i+1], 'pos y': y[start:i+1], 'ori': ori[start:i+1], 'frame': frame[start:i+1]})
            tracklets.append(t)
            start = i+1
        t = pd.DataFrame({'pos x': x[start:], 'pos y': y[start:], 'ori': ori[start:], 'frame': frame[start:]})
        tracklets.append(t)

    # filter out tracklets with less than 5*FPS frames
    tracklets = [t for t in tracklets if len(t) >= 5*FPS]

    # add ring and yang coordinates to each tracklet
    i = 0
    for t in tqdm(tracklets):
        t['big ring pos'], t['big ring dist'] = zip(*[ring_coordinates([x, y], big_ring_props) for x, y in zip(t['pos x'], t['pos y'])])
        t['small ring pos'], t['small ring dist'] = zip(*[ring_coordinates([x, y], small_ring_props) for x, y in zip(t['pos x'], t['pos y'])])
        t['track id'] = i
        i += 1

    # combine and save tracklets
    tracklets = pd.concat(tracklets)
    tracklets.to_csv(f'phase_{phase}_tracklets.csv', index=False)



100%|██████████| 23/23 [00:01<00:00, 21.59it/s]


In [15]:
# plot all tracklets in phases
for phase in [1,2,3]:
    tracklets = pd.read_csv(f'phase_{phase}_tracklets.csv')
    fig, ax = plt.subplots()
    for i in range(tracklets['track id'].max()):
        t = tracklets[tracklets['track id'] == i]
        plt.plot(t['pos x'], t['pos y'], 'w-', lw=0.2)
    if phase == 1:
        draw_ring(big_ring_props, axis=ax, color='r', bg=p1_bg)
    elif phase == 2:
        draw_ring(big_ring_props, axis=ax, color='k', bg=p2_bg)
    else:
        draw_ring(small_ring_props, axis=ax, color='r', bg=p3_bg)
    plt.gca().set_aspect('equal')
    plt.title(f'Phase {phase}')
    # save figure
    plt.savefig(f'figures/phase_{phase}_tracklets.png')
    plt.show()

In [16]:
# plot all start and end points of tracklets
for phase in [1,2,3]:
    tracklets = pd.read_csv(f'phase_{phase}_tracklets.csv')
    fig, ax = plt.subplots(1, 2)
    # draw rings
    if phase == 1:
        draw_ring(big_ring_props, axis=ax[0], color='r', bg=p1_bg)
        draw_ring(big_ring_props, axis=ax[1], color='r', bg=p1_bg)
    elif phase == 2:
        draw_ring(big_ring_props, axis=ax[0], color='k', bg=p2_bg)
        draw_ring(big_ring_props, axis=ax[1], color='k', bg=p2_bg)
    else:
        draw_ring(small_ring_props, axis=ax[0], color='r', bg=p3_bg)
        draw_ring(small_ring_props, axis=ax[1], color='r', bg=p3_bg)
    for i in range(tracklets['track id'].max()):
        t = tracklets[tracklets['track id'] == i]
        ax[0].scatter(t.iloc[0]['pos x'], t.iloc[0]['pos y'], color=plt.cm.rainbow(i/tracklets['track id'].max()), s=1)
        ax[1].scatter(t.iloc[-1]['pos x'], t.iloc[-1]['pos y'], color=plt.cm.rainbow(i/tracklets['track id'].max()), s=1)
    ax[0].set_title('Start points')
    ax[1].set_title('End points')
    ax[0].set_aspect('equal')
    ax[1].set_aspect('equal')
    plt.suptitle(f'Phase {phase}')
    plt.tight_layout()
    # save figure
    plt.savefig(f'figures/phase_{phase}_start_end.png')
    plt.show()


In [12]:
encounter_distance = 5*2 # mm body width
encounter_distance = encounter_distance * sf # scale to pixels
encounter_distance = encounter_distance / 2 # convert distance
print(f'Encounter distance: {encounter_distance} pixels = {encounter_distance/sf:.2f} mm = {encounter_distance/sf*0.0393701:.2f} inches')

trail_width = big_ring_props['trail_width']/2
print(f'BIG Trail width: {trail_width} pixels = {trail_width/sf:.2f} mm = {trail_width/sf*0.0393701:.2f} inches')

trail_width = small_ring_props['trail_width']/2
print(f'SMALL Trail width: {trail_width} pixels = {trail_width/sf:.2f} mm = {trail_width/sf*0.0393701:.2f} inches')

Encounter distance: 27.605768634807756 pixels = 5.00 mm = 0.20 inches
BIG Trail width: 3.639969337419302 pixels = 0.66 mm = 0.03 inches
SMALL Trail width: 3.791565000089264 pixels = 0.69 mm = 0.03 inches


In [18]:
# plot every point within the target distance of the rings
fig, ax = plt.subplots()
plt.imshow(p2_bg, cmap='gray')

# create a meshgrid
res = 50
x = np.linspace(0, p2_bg.shape[1], res)
y = np.linspace(0, p2_bg.shape[0], res)
X, Y = np.meshgrid(x, y)
X = X.flatten()
Y = Y.flatten()
points = np.vstack([X, Y]).T

for point in tqdm(points):
    pos, dist = ring_coordinates(point, big_ring_props)
    if abs(dist) < encounter_distance + trail_width:
        plt.scatter(point[0], point[1], color=plt.cm.cool(pos/(2*np.pi)), s=1)


for point in tqdm(points):
    pos, dist = ring_coordinates(point, small_ring_props)
    if abs(dist) < 20:
        plt.scatter(point[0], point[1], color=plt.cm.cool(pos/(2*np.pi)), s=1)

plt.title('Points within the target distance of the rings')

draw_ring(big_ring_props, axis=ax, color='k')
draw_ring(small_ring_props, axis=ax, color='k')

# add colorbar
sm = plt.cm.ScalarMappable(cmap=plt.cm.cool, norm=plt.Normalize(0, 2*np.pi))
sm.set_array([])
plt.colorbar(sm, ticks=[0, np.pi, 2*np.pi], label='Angle', orientation='horizontal', ax=ax)
# save figure
plt.savefig('figures/targeted_points.png')
plt.show()



100%|██████████| 2500/2500 [00:00<00:00, 13061.79it/s]


#### TEST TRACKLET

In [19]:
tracklets = pd.read_csv('phase_2_tracklets.csv')
tracklets = [group for _, group in tracklets.groupby('track id')]

tracklet = tracklets[0]
ring = 'big'
plt.figure()
plt.plot(tracklet['big ring dist'].values, np.unwrap(tracklet['big ring pos'].values), 'k-', lw=0.2)
plt.show()

val = np.abs(np.concatenate([[np.inf], tracklet[ring+' ring dist'].values])) < encounter_distance + trail_width
count = np.cumsum(np.concatenate([np.array([np.nan]),np.diff(val.astype(int))])>0)
mes = val*count
plt.figure()
plt.plot(tracklet['pos x'], tracklet['pos y'], 'k-', lw=0.2)
draw_ring(big_ring_props, color='k', axis=plt.gca())
# draw encounters with scatter
plt.scatter(tracklet['pos x'][mes[1:]>0], tracklet['pos y'][mes[1:]>0], c=mes[mes>0], cmap='viridis')
plt.gca().set_aspect('equal')
plt.show()

In [13]:
def get_crossings(encounter, ring, min_deviation):
    crossings = np.abs(np.diff(np.sign(encounter[ring+' ring dist'])))
    # distance traveled between crossings must be atleast the trail width
    encounters_between_crossings = []
    crossing_points = np.where(crossings > 0)[0]
    if len(crossing_points) == 0:
        return 0, []
    encounters_between_crossings.append(encounter[:crossing_points[0]])
    for i in range(len(crossing_points)-1):
        encounters_between_crossings.append(encounter[crossing_points[i]:crossing_points[i+1]])
    # get the maximum distance travelled away from the center
    furthest_distance = np.array([np.max(np.abs(encounter[ring+' ring dist'])) for encounter in encounters_between_crossings])
    # get the crossing points where the distance travelled is atleast the trail width
    crossing_points = np.array(crossing_points)[np.where(furthest_distance > min_deviation)[0]]
    n_crossings = len(crossing_points)
    return n_crossings, crossing_points


N_CROSSINGS_FOR_ENCOUNTER = 1
# create a subfolder for N_CROSSINGS_FOR_ENCOUNTER
if not os.path.exists(f'figures/Crossings_{N_CROSSINGS_FOR_ENCOUNTER}'):
    os.makedirs(f'figures/Crossings_{N_CROSSINGS_FOR_ENCOUNTER}')
if not os.path.exists(f'videos/Crossings_{N_CROSSINGS_FOR_ENCOUNTER}'):
    os.makedirs(f'videos/Crossings_{N_CROSSINGS_FOR_ENCOUNTER}')
if not os.path.exists(f'Crossings_{N_CROSSINGS_FOR_ENCOUNTER}'):
    os.makedirs(f'Crossings_{N_CROSSINGS_FOR_ENCOUNTER}')

In [14]:

# get encounters in each phase
for phase in [1,2,3]:
    tracklets = pd.read_csv(f'phase_{phase}_tracklets.csv')
    tracklets = [group for _, group in tracklets.groupby('track id')]

    if phase == 1:
        props = big_ring_props
        ring = 'big'
        trail_width = big_ring_props['trail_width']/2
    elif phase == 2:
        props = big_ring_props
        ring = 'big'
        trail_width = big_ring_props['trail_width']/2
    else:
        props = small_ring_props
        ring = 'small'
        trail_width = small_ring_props['trail_width']/2
    
    print(f'Phase {phase}: Encounter distance: {encounter_distance} pixels, Trail width: {trail_width} pixels')
    # get encounters
    encounters = []
    for tracklet in tracklets:
        val = np.abs(np.concatenate([[np.inf], tracklet[ring+' ring dist'].values])) < encounter_distance + trail_width
        count = np.cumsum(np.concatenate([np.array([np.nan]),np.diff(val.astype(int))])>0)
        mes = val*count
        
        # get every continuous segment where the fly is within encounter_distance of the ring circle
        encounters += [group for m, group in tracklet.groupby(mes[1:]) if m != 0]
    # filter for encounters with atleast 1 sign change in the ring position
    n_crossings= [get_crossings(encounter, ring, trail_width)[0] for encounter in encounters]
    mask = [n_crossings[i] >= N_CROSSINGS_FOR_ENCOUNTER for i in range(len(encounters))]
    encounters = [encounter for i,encounter in enumerate(encounters) if mask[i]]
    n_crossings = [n_crossings[i] for i in range(len(n_crossings)) if mask[i]]

    print(f'Phase {phase}: {len(encounters)} encounters')
    
    # plot all encounters
    fig, ax = plt.subplots()
    for encounter in encounters:
        plt.plot(encounter['pos x'], encounter['pos y'], '-', lw=0.5, color=plt.cm.cool(np.random.rand()))
    if phase == 1:
        draw_ring(big_ring_props, axis=ax, color='r', bg=p1_bg)
    elif phase == 2:
        draw_ring(big_ring_props, axis=ax, color='k', bg=p2_bg)
    else:
        draw_ring(small_ring_props, axis=ax, color='r', bg=p3_bg)
    plt.gca().set_aspect('equal')
    plt.title(f'Phase {phase}')
    # save figure
    plt.savefig(f'figures/Crossings_{N_CROSSINGS_FOR_ENCOUNTER}/phase_{phase}_encounters.png')
    plt.show()

    # plot in ring space
    fig, ax = plt.subplots()

    max_distances = []
    for encounter in encounters:
        position = (np.unwrap(encounter[ring+' ring pos'])*props['radius']/sf)
        # find point of first crossing
        crossing = np.argmax(np.abs(np.diff(np.sign(encounter[ring+' ring dist'].values))))
        position = position - position[crossing]
        dists = [np.abs(np.max(position[crossing:])),np.abs(np.min(position[crossing:]))]
        if np.argmax(dists) == 1:
            position = -position
        distance = encounter[ring+' ring dist']/sf
        max_distances.append(np.max(dists))
        ax.plot(distance, position, 'r-', lw=1, alpha=0.5)
        # plot the trail width
    ax.axvline(-trail_width/sf, color='k', alpha=0.5, ls='--')
    ax.axvline(trail_width/sf, color='k', alpha=0.5, ls='--')


    ax.set_xlabel('Distance from ring center (mm)')
    ax.set_ylabel('Position on ring (mm)')
    ax.set_title(f'Phase {phase}')
    ax.set_aspect('equal')
    # save figure
    plt.savefig(f'figures/Crossings_{N_CROSSINGS_FOR_ENCOUNTER}/phase_{phase}_encounters_unwrapped.png')
    plt.show()

    # sort by max distance
    encounters = [encounters[i].reset_index() for i in np.argsort(max_distances)]
    n_crossings = [n_crossings[i] for i in np.argsort(max_distances)]
    max_distances = np.sort(max_distances)

    # # sort by number of crossings
    # encounters = [encounters[i] for i in np.argsort(n_crossings)]
    # max_distances = [max_distances[i] for i in np.argsort(n_crossings)]
    # n_crossings = np.sort(n_crossings)

    # save ring encounters after adding an id
    for i, encounter in enumerate(encounters):
        encounter['id'] = i
        encounter['max_distance'] = max_distances[i]
        encounter['n_crossings'] = n_crossings[i]
    ring_encounters_CSV = pd.concat(encounters) if len(encounters) > 0 else pd.DataFrame(columns=['id', 'max_distance', 'n_crossings'])
    ring_encounters_CSV.to_csv(f'Crossings_{N_CROSSINGS_FOR_ENCOUNTER}/encounters_phase_{phase}.csv', index=False)

    # plot the distribution of max distances
    plt.figure()
    plt.hist(max_distances, bins=20, density=True)
    plt.xlabel('Max distance (mm)')
    plt.ylabel('Frequency')
    plt.title(f'Phase {phase}')
    # save figure
    plt.savefig(f'figures/Crossings_{N_CROSSINGS_FOR_ENCOUNTER}/phase_{phase}_max_distances.png')
    plt.show()
        

Phase 1: Encounter distance: 27.605768634807756 pixels, Trail width: 3.639969337419302 pixels
Phase 1: 1093 encounters
Phase 2: Encounter distance: 27.605768634807756 pixels, Trail width: 3.639969337419302 pixels
Phase 2: 327 encounters
Phase 3: Encounter distance: 27.605768634807756 pixels, Trail width: 3.791565000089264 pixels
Phase 3: 408 encounters


In [54]:
for phase in [1,2,3]:
    if phase == 1:
        props = big_ring_props
        ring = 'big'
        trail_width = big_ring_props['trail_width']/2
    elif phase == 2:
        props = big_ring_props
        ring = 'big'
        trail_width = big_ring_props['trail_width']/2
    else:
        props = small_ring_props
        ring = 'small'
        trail_width = small_ring_props['trail_width']/2

    # get ring encounters for phase 1
    encounters = pd.read_csv(f'Crossings_{N_CROSSINGS_FOR_ENCOUNTER}/encounters_phase_{phase}.csv')
    if len(encounters) == 0:
        print(f'No encounters in phase {phase}')
        continue
    encounters = [group.reset_index(drop=True) for _, group in encounters.groupby('id')]
    max_distances = [encounter['max_distance'].values[0] for encounter in encounters]
    n_crossings = [encounter['n_crossings'].values[0] for encounter in encounters]

    # plot the top 10 in different plots (ring space)
    fig, ax = plt.subplots(2, 5, sharex=True, sharey=True)
    for i, encounter in enumerate(encounters[-10:][::-1]):
        position = (np.unwrap(encounter[ring+' ring pos'])*props['radius']/sf)
        # find point of first crossing
        _, crossings = get_crossings(encounter, ring, trail_width)
        crossing = crossings[0]
        position = position - position[crossing]
        dists = [np.abs(np.max(position[crossing:])),np.abs(np.min(position[crossing:]))]
        if np.argmax(dists) == 1:
            position = -position
        distance = encounter[ring+' ring dist']/sf
        ax.flatten()[i].plot(distance, position, 'r-', lw=1, alpha=0.5)
        ax.flatten()[i].set_title(f'Encounter {i+1}')
        ax.flatten()[i].set_aspect('equal')
        # draw the trail width
        ax.flatten()[i].axvline(-trail_width/sf, color='k', ls='--', lw=0.5)
        ax.flatten()[i].axvline(trail_width/sf, color='k', ls='--', lw=0.5)
    # hide axis with no plot
    for i in range(10-len(encounters)):
        ax.flatten()[-i-1].axis('off')
    plt.suptitle(f'Top 10 encounters in phase {phase}')
    plt.tight_layout()
    plt.savefig(f'figures/Crossings_{N_CROSSINGS_FOR_ENCOUNTER}/top_10_encounters_phase_{phase}_unwrapped.png')
    plt.show()

    # plot the top 10 in different plots (xy space)
    fig, ax = plt.subplots(2, 5, sharex=True, sharey=True)
    for i, encounter in enumerate(encounters[-10:][::-1]):
        ax.flatten()[i].plot(encounter['pos x'], encounter['pos y'], 'w-', lw=0.5)
        ax.flatten()[i].set_title(f'Encounter {i+1}')
        ax.flatten()[i].set_aspect('equal')
        draw_ring(props, color='k', axis=ax.flatten()[i])
    # hide axis with no plot
    for i in range(10-len(encounters)):
        ax.flatten()[-i-1].axis('off')
    plt.suptitle(f'Top 10 encounters in phase {phase}')
    plt.tight_layout()
    plt.savefig(f'figures/Crossings_{N_CROSSINGS_FOR_ENCOUNTER}/top_10_encounters_phase_{phase}.png')
    plt.show()


In [35]:
# create a joint video of all top 10 encounters
import skvideo.io
import os
from tqdm import tqdm

for phase in [1,2,3]:

    if phase == 1:
        props = big_ring_props
        ring = 'big'
        trail_width = big_ring_props['trail_width']/2
        bg = p1_bg
    elif phase == 2:
        props = big_ring_props
        ring = 'big'
        trail_width = big_ring_props['trail_width']/2
        bg = p2_bg
    else:
        props = small_ring_props
        ring = 'small'
        trail_width = small_ring_props['trail_width']/2
        bg = p3_bg

    encounters = pd.read_csv(f'Crossings_{N_CROSSINGS_FOR_ENCOUNTER}/encounters_phase_{phase}.csv')
    if len(encounters) == 0:
        print(f'No encounters in phase {phase}')
        continue
    encounters = [group.reset_index(drop=True) for _, group in encounters.groupby('id')]
    # reverse the order
    encounters = encounters[::-1]

    xs = [encounter['pos x'].values for encounter in encounters]
    ys = [encounter['pos y'].values for encounter in encounters]
    on_trail = [encounter[ring+' ring dist'].values<trail_width for encounter in encounters]

    # get background image
    bg = cv2.cvtColor((bg*255).astype(np.uint8), cv2.COLOR_GRAY2RGB)
    width, height = bg.shape[1], bg.shape[0]


    writer = skvideo.io.FFmpegWriter(f'videos/Crossings_{N_CROSSINGS_FOR_ENCOUNTER}/top_10_encounters_phase_{phase}.mp4', outputdict={'-r': str(FPS)})
    for j in tqdm(range(10)):
        if j >= len(encounters):
            break
        for i in range(len(encounters[j])):
            frame = bg.copy()
            if on_trail[j][i]:
                color = (255, 0, 0)
            else:
                color = (255, 255, 255)
            # draw a trail
            for k in range(i):
                if on_trail[j][k]:
                    cv2.line(frame, (int(xs[j][k]), int(ys[j][k])), (int(xs[j][k+1]), int(ys[j][k+1])), (255, 0, 0), 1)
                else:
                    cv2.line(frame, (int(xs[j][k]), int(ys[j][k])), (int(xs[j][k+1]), int(ys[j][k+1])), (255, 255, 255), 1)
            cv2.circle(frame, (int(xs[j][i]), int(ys[j][i])), 5, color, -1)
            # add text
            cv2.putText(frame, f'Encounter {j+1}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1)
            cv2.putText(frame, f'Time: {i/FPS:.2f} s', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            writer.writeFrame(frame)
        # wait for 2 second at the end of each encounter
        for _ in range(2*FPS):
            writer.writeFrame(frame)
    writer.close()



  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:04<00:00,  2.37it/s]


No encounters in phase 2


 80%|████████  | 8/10 [00:05<00:01,  1.35it/s]


In [77]:
encounters[-1]

,index,pos x,pos y,ori,frame,big ring pos,big ring dist,small ring pos,small ring dist,track id,id,max_distance,n_crossings
0,7229,569.78,489.44,2.9100,7229,0.332485,-96.797523,0.370520,31.016715,0,407,93.858885,8
1,7230,555.40,484.30,2.7664,7230,0.331250,-112.067548,0.373561,15.751991,0,407,93.858885,8
2,7231,548.50,477.50,2.5186,7231,0.297743,-120.733832,0.343743,6.903166,0,407,93.858885,8
3,7232,539.83,473.04,2.5943,7232,0.282832,-130.317936,0.332940,-2.755819,0,407,93.858885,8
4,7233,539.20,471.83,2.5344,7233,0.274215,-131.256339,0.324943,-3.742929,0,407,93.858885,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,7389,320.02,502.93,-1.2311,7389,2.623879,-120.235663,2.576450,6.978932,0,407,93.858885,8
161,7390,322.66,512.46,-1.2726,7390,2.549046,-117.454446,2.504259,10.194762,0,407,93.858885,8
162,7391,324.18,520.48,-1.3323,7391,2.492043,-114.022477,2.449761,13.945828,0,407,93.858885,8
163,7392,324.77,530.87,-1.3491,7392,2.429348,-107.937515,2.390461,20.368234,0,407,93.858885,8


In [79]:
plt.figure()
encounter = encounters[-1]
orientation = (encounter['ori'].values % (2*np.pi) - encounter[ring+' ring pos'].values % (2*np.pi)) % (2*np.pi)
orientation = np.unwrap(orientation)
# find point of first crossing
_, crossings = get_crossings(encounter, ring, trail_width)
crossing = crossings[0]
# plot the orientation
plt.plot(orientation)
plt.axvline(crossing, color='r', ls='--')
plt.title('Orientation')
plt.show()

In [ ]:
# get the headings right before crossing for all flies


In [47]:
fly_body_radius = 0.5 # mm
fly_body_radius = fly_body_radius * sf # scale to pixels
fly_body_radius = fly_body_radius / 2 # convert distance

In [50]:
def get_entry_exit(encounter, ring, fly_body_radius=fly_body_radius):
    # # interaction with outer ring
    # outer_ring_crossings = np.abs(np.diff(np.sign(encounter[ring+' ring dist']+trail_width-fly_body_radius)))
    # # interaction with inner ring
    # inner_ring_crossings = np.abs(np.diff(np.sign(encounter[ring+' ring dist']-trail_width+fly_body_radius)))
    # find every event where the fly enters the trail from outer edge
    distance_from_outer_ring = encounter[ring+' ring dist']+trail_width-fly_body_radius # adjusted for body position
    entry_outer = np.where(np.diff(np.sign(distance_from_outer_ring)) < 0)[0]
    exit_outer = np.where(np.diff(np.sign(distance_from_outer_ring)) > 0)[0]
    # find every event where the fly enters the trail from inner edge
    distance_from_inner_ring = encounter[ring+' ring dist']-trail_width+fly_body_radius # adjusted for body position
    entry_inner = np.where(np.diff(np.sign(distance_from_inner_ring)) > 0)[0]
    exit_inner = np.where(np.diff(np.sign(distance_from_inner_ring)) < 0)[0]
    # get the entry and exit points (combine inner and outer)
    entry = np.concatenate([entry_outer, entry_inner])
    exit = np.concatenate([exit_outer, exit_inner])
    # sort the entry and exit points
    entry = np.sort(entry)
    exit = np.sort(exit)
    # plot the entry and exit points
    # draw the ring
    if ring == 'big':
        draw_ring(big_ring_props, color='g')
    else:
        draw_ring(small_ring_props, color='g')
    plt.plot(encounter['pos x'], encounter['pos y'], 'w-', lw=0.2)
    plt.scatter(encounter.iloc[entry]['pos x'], encounter.iloc[entry]['pos y'], c='r', label='Entry')
    plt.scatter(encounter.iloc[exit]['pos x'], encounter.iloc[exit]['pos y'], c='b', label='Exit')
    plt.legend()
    plt.gca().set_aspect('equal')
    plt.title('Entry and exit points')
    plt.show()
    return entry, exit

get_entry_exit(encounters[-1], 'small')

(array([  2,   8,  17,  56, 120, 147, 150, 155]),
 array([  2,   5,  12,  55,  75, 146, 149, 154]))

In [48]:
trail = p3_bg-p2_bg
# set all negative to zero
trail[trail < 0] = 0
r = 0.015
# set everything between -min_val and min_val to zero
trail[trail < r] = 0
# set everything else to 1
trail[trail != 0] = 1
# apply morphological operations
import cv2
kernel = np.ones((5,5),np.uint8)
trail = cv2.morphologyEx(trail, cv2.MORPH_CLOSE, kernel)
# keep only the single largest connected component
n, labels = cv2.connectedComponents(trail.astype(np.uint8))
sizes = np.bincount(labels.flatten())
trail = (labels == np.argmax(sizes[1:])+1).astype(np.uint8)
# dilate the trail
trail = cv2.dilate(trail, kernel, iterations=1)
# erode the trail
trail = cv2.erode(trail, kernel, iterations=1)


# show the trail
plt.figure()
plt.imshow(trail + p2_bg, cmap='gray')
plt.title('Trail')
plt.show()

In [50]:
# for every point in the encounter estimate the intensity of the trail via interpolation

# 2d interpolation of the trail
x = np.arange(trail.shape[1])
y = np.arange(trail.shape[0])
X, Y = np.meshgrid(x, y)
points = np.vstack([X.flatten(), Y.flatten()]).T
values = trail.flatten()
from scipy.interpolate import griddata





In [52]:
# increase resolution of encounter by 10 times
encounter = encounters[-1]
upsample = 10
new_encounter = pd.DataFrame()
for col in encounter.columns:
    new_encounter[col] = np.interp(np.linspace(0, len(encounter), upsample*len(encounter)), np.arange(len(encounter)), encounter[col])
new_encounter = new_encounter.reset_index(drop=True)
new_encounter.head()

,index,pos x,pos y,ori,frame,big ring pos,big ring dist,small ring pos,small ring dist,track id,id,max_distance,n_crossings
0,7229.000000,569.780000,489.440000,2.910000,7229.000000,0.332485,-96.797523,0.370520,31.016715,0.0,407.0,93.858885,8.0
1,7229.100061,568.341128,488.925688,2.895631,7229.100061,0.332361,-98.325451,0.370825,29.489317,0.0,407.0,93.858885,8.0
2,7229.200121,566.902256,488.411377,2.881263,7229.200121,0.332238,-99.853380,0.371129,27.961919,0.0,407.0,93.858885,8.0
3,7229.300182,565.463384,487.897065,2.866894,7229.300182,0.332114,-101.381308,0.371433,26.434521,0.0,407.0,93.858885,8.0
4,7229.400243,564.024512,487.382753,2.852525,7229.400243,0.331991,-102.909237,0.371737,24.907123,0.0,407.0,93.858885,8.0


In [53]:
intensity = griddata(points, values, (new_encounter['pos x'], new_encounter['pos y']), method='linear')
# plot the intensity
plt.figure()
plt.plot(intensity)
plt.title('Trail intensity')
plt.show()



In [56]:
# plot the encounters with the trail intensity
plt.figure()
plt.imshow(p3_bg, cmap='gray')
plt.plot(new_encounter['pos x'], new_encounter['pos y'], 'w-', lw=0.2)
plt.scatter(new_encounter['pos x'], new_encounter['pos y'], c=intensity, cmap='viridis', s=1)
plt.colorbar(label='Trail intensity')
plt.gca().set_aspect('equal')
plt.show()